# Titanic Survival Prediction

<img src="flow_chart.png" height=200px width=800px></img>

## Frame the problem

The sinking of the RMS Titanic is one of the most infamous shipwrecks in history.  On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.

One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.

In this challenge, we ask you to complete the analysis of what sorts of people were likely to survive. In particular, we ask you to apply the tools of machine learning to predict which passengers survived the tragedy.

## Obtain Data

#### Importing the basic required libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as ms
%matplotlib inline

### Reading the data from CSV file

In [ ]:
data = pd.read_csv('titanic.csv')

## Analyze Data

#### Obtaining a glimpse of data

In [ ]:
data.head(3)

In [ ]:
data.tail(3)

In [ ]:
type(data)

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.describe()

#### Visualization of the Data

In [ ]:
ms.matrix(data)

In [ ]:
data.info()

In [ ]:
sns.jointplot(x='Fare',y='Age',data=data)

In [ ]:
sns.distplot(data['Fare'])

In [ ]:
data.corr()

In [ ]:
sns.heatmap(data.corr(),cmap='coolwarm',xticklabels=True,annot=True)
plt.title('data.corr()')

In [ ]:
sns.swarmplot
sns.swarmplot(x='Pclass',y='Age',data=data,palette='Set2')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',data=data,palette='RdBu_r')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Sex',data=data,palette='RdBu_r')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Pclass',data = data,palette='rainbow')

In [ ]:
data['Age'].hist(bins = 40, color = 'purple', alpha = 0.6)

In [ ]:
sns.countplot(x = 'SibSp', data = data)

In [ ]:
data['Fare'].hist(color = 'green', bins = 40, figsize = (8,3))

## Feature Engineering

#### Fill the missing values in the obtained data

In [ ]:
plt.figure(figsize=(12, 7))
sns.boxplot(x='Pclass',y='Age',data=data,palette='winter')

In [ ]:
data.groupby('Pclass')['Age'].median()



The average age for each of the classes are estimated to be as follows:
  
  * For **Class 1** - The median age is 37
  * For **Class 2** - The median age is 29
  * For **Class 3** - The median age is 24
  
Let's impute these values into the age column.



In [ ]:
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):
        # Class-1
        if Pclass == 1:
            return 37
        # Class-2 
        elif Pclass == 2:
            return 29
        # Class-3
        else:
            return 24

    else:
        return Age



Applying the function.

In [ ]:
data['Age'] = data[['Age','Pclass']].apply(impute_age,axis=1)

Now let's visualize the missing values.

In [ ]:
ms.matrix(data)

The Age column is imputed sucessfully.

Let's drop the Cabin column and the row in the Embarked that is NaN.

In [ ]:
data.drop('Cabin', axis = 1,inplace=True)

In [ ]:
data.head()

In [ ]:
data.dropna(inplace = True)

In [ ]:
ms.matrix(data)

In [ ]:
data.info()

### Categorical value conversion

In [ ]:
data.info()

In [ ]:
data['Sex'].unique()

In [ ]:
data['Sex'].value_counts()

In [ ]:
sex_df = pd.get_dummies(data['Sex'],drop_first=3)
sex_df.head()

In [ ]:
data['Embarked'].unique()

In [ ]:
data['Embarked'].value_counts()

In [ ]:
644+168+77

In [ ]:
embark_df = pd.get_dummies(data['Embarked'],drop_first=True)
embark_df.head()

In [ ]:
old_data = data.copy()
data.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)
data.head()

In [ ]:
old_data.info()

In [ ]:
data = pd.concat([data,sex_df,embark_df],axis=1)

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

## Model Selection

### Train-Test Split

<img src="supervised_flow_chart.png" height = 200px width = 800px></img>

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data.drop('Survived',axis=1), 
                                                    data['Survived'], test_size=0.30, 
                                                    random_state=101)

### Logistic Regression

#### Training the model

In [ ]:
from sklearn.linear_model import LogisticRegression

# Build the Model.
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

### Predicting the model on the test set

In [ ]:
predicted =  logmodel.predict(X_test)

In [ ]:
predicted

## Tune the Model

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

### Confusion Matrix

True positive   |	False positive,  
________________|________________  
                |                  
False negative  |	True negative    `

In [ ]:
print(confusion_matrix(y_test, predicted))

#### Precision Score

The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.

The best value is 1 and the worst value is 0.



In [ ]:
from sklearn.metrics import precision_score

print(precision_score(y_test,predicted))

#### Recall score

The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

The best value is 1 and the worst value is 0.



In [ ]:
from sklearn.metrics import recall_score

print(recall_score(y_test,predicted))



#### f1_score

The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the F1 score are equal. The formula for the F1 score is:
                F1 = 2 \* (precision \* recall) / (precision + recall)

In [ ]:
from sklearn.metrics import f1_score

print(f1_score(y_test,predicted))

To get all the above metrics at one go, use the following function:

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test,predicted))

## Predicting on New Class

In [ ]:
prod_data=pd.read_csv('production.csv')

In [ ]:
prod_data.info()

In [ ]:
prod_data.head()

In [ ]:
ms.matrix(prod_data)

### Data Cleaning

In [ ]:
prod_data['Age'] = prod_data[['Age','Pclass']].apply(impute_age,axis=1)

ms.matrix(prod_data)

prod_data.drop('Cabin', axis = 1, inplace= True)

ms.matrix(prod_data)

prod_data.fillna(prod_data['Fare'].mean(),inplace=True)

prod_data.info()

ms.matrix(prod_data)

sex = pd.get_dummies(prod_data['Sex'], drop_first=True)
embark = pd.get_dummies(prod_data['Embarked'], drop_first=True)

prod_data.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)

prod_data = pd.concat([prod_data,sex,embark],axis=1)

prod_data.head()

### Predicting on New Dataset

In [ ]:
predict1=logmodel.predict(prod_data)

predict1

df1=pd.DataFrame(predict1,columns=['Survived'])

df2=pd.DataFrame(prod_data['PassengerId'],columns=['PassengerId'])

df2.head()

result = pd.concat([df2,df1],axis=1)
result.head()

In [1]:
5
6

6

In [ ]:
result.tail()

### Writing to CSV File

In [ ]:
result.to_csv('result.csv',index=False)

---
                                    THE END